# Lane Detection and Object Recognition
To build a self-driving car, the perception is a very important part. Like human drivers, the computers on self-driving cars need to know the surrounding environment. In this MP, with the help from powerful open source frameworks like OpenCV, we will achieve offline lane detection. This lab will emphasize on traditional robotics approach, to give you a glimpse of the how self-driving car engineers tried to achieve computer vision before machine learning and GPU came into play. And many of those techniques are still widely used in the latest autonomous car systems.<br>  
### You only need to change the code between '## TODO' and '####' 
You can make whatever changes you want to improve the performance. We will grade the MP based on your results, not how you achieve it. BUT make sure to explain how your method works in your report. 

In [2]:
# import numpy as np
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

print ('...')

...


We put 6 images in folder ./RightHookTestImage. You can pick one test image from them.

(1080, 1920, 3)


In [3]:
testImageRoot = "./RightHookTestImage/rh3.png"

rawImg = cv2.imread(testImageRoot)
print (rawImg.shape)
#Note that cv2 will read image color channel in BGR order. 
rawImg = cv2.cvtColor(rawImg,cv2.COLOR_BGR2RGB)
plt.imshow(rawImg)
plt.show()

To find the lanes, we wish to get rid of extra part in the image, like the sky, trees, and other cars. It is much easier to find the lanes from a bird view than from a normal driver's view. Here we just assume that we are on a flat road. We need to use cv2.line to find the location of 4 points on the original image and map the same 4 points on the bird view. The points we offered are just a good starting point. You can definitely find something better. <br> 
Read this useful document first:<br> https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html

Your result is supposed to be like:
<img src="./ResultImages/1.png">


In [ ]:
rawImage = cv2.imread(testImageRoot)
#try to find 4 reference points
##TO DO
x1, y1 = 
x2, y2 = 
x3, y3 = 
x4, y4 = 
####

cv2.circle(rawImage,(x1,y1), 10, (0,0,255), -1)
cv2.circle(rawImage,(x2,y2), 10, (0,0,255), -1)
cv2.circle(rawImage,(x3,y3), 10, (0,0,255), -1)
cv2.circle(rawImage,(x4,y4), 10, (0,0,255), -1)

#plot 4 points on original image
plot1 = cv2.cvtColor(rawImage,cv2.COLOR_BGR2RGB)
plt.imshow(plot1)
plt.show()

In [4]:
def warpImage(img):
#(1) use cv2.getPerspectiveTransform() to get M, the transform matrix, and Minv, the inverse
#(2) use cv2.warpPerspective to generate warped image in bird view, 
    ## TODO    
    h,w = img.shape[:2] #width and height
    #put the 4 points you choose in last cell here 
    x1, y1 = 
    x2, y2 = 
    x3, y3 = 
    x4, y4 = 
    ####
    
    #destination points on bird view image
    dst = np.float32([(600,0),
                  (w-600,0),
                  (600,h),
                  (w-600,h)])   
    src = np.float32([(x2,y2),(x4,y4),(x1,y1),(x3,y3)])

    
    # use cv2.getPerspectiveTransform() to get M, the transform matrix, and Minv, the inverse
    ## TODO
    M = 
    Minv = 
    ####
    
    # use cv2.warpPerspective to transform to bird view
    warpedImg = cv2.warpPerspective(img, M, (w,h), flags=cv2.INTER_LINEAR)

    return warpedImg, M, Minv

In [7]:
##test warpImage
rawImage = cv2.imread(testImageRoot)
warpedImage, M, Minv = warpImage(rawImage)

plot2 = cv2.cvtColor(warpedImage,cv2.COLOR_BGR2RGB)
plt.imshow(plot2)
plt.show()

## Color Space and Gradient Threshold For Edge Detection
In this part, you will implement multiple methods to do edge detection and distinguish the lanes pixels from the background. And you can choose whatever combinations you want to add into the image processing pipeline. We have also offered you a nice piece of test bench. Here are a few useful tips:<br>
1. Convert image to gray scale use cv2.cvtColor for all gradient methods.
2. Use image blurring to smooth over the shift caused by vibration of camera.
3. In color space method, we can assume lanes are yellow or white. So just mask the image with white and yellow.

First use Laplacian operator, which calculate the second order derievative in one single pass to detect edges.
Read the tutorial before you start: https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html

In [5]:
def laplacian(img, thresh_min=0, thresh_max=255): #after you find the correct values, use them as default values  
    #1. convert the image to gray scale
    #2. Gaussian blur the image
    #3. implement laplacian edge detetion using openCV
    #4. using numpy to scale the result to uint8(0-255)
    #5. using the threshold to create a binary image and return it
    
    ## TODO
    
    # name your output binary image to be laplacian
    laplacian = 
    
    ####
    
    scaled_lap = np.uint8(255*combined/np.max(laplacian))
    binary_output = np.zeros_like(scaled_lap)
    binary_output[(scaled_lap >= thresh_min) & (scaled_lap <= thresh_max)] = 1
    
    return binary_output
print('...')

...


Then use Sobel operator, which calculate the first derivative separatly for X and Y axis. Unlike previous part, we need to apply it individually on X and Y axis. Then combine them in to one binary image output.<br>
Follow this useful tutorial: https://docs.opencv.org/2.4/doc/tutorials/imgproc/imgtrans/sobel_derivatives/sobel_derivatives.html

In [2]:
def sobel(img, thresh_min=0, thresh_max=255): # use the threshold values you found to replace 0 and 255
    #1. convert the image to gray scale
    #2. Gaussian blur the image
    #3. Use Sobel to find derievatives for both X and Y Axis
    #4. Use addweight to combine the results
    #5. Convert each pixel to unint8, then apply threshold to get binary image
    ## TODO

    #name your result to be sobelX and sobelY
    sobelX = 
    sobelY = 
    
    ####
    
    combined = cv2.addWeighted(sobelX, 0.7, sobelY, 0.3, 0)
    scaled_lap = np.uint8(255*combined/np.max(combined))
    binary_output = np.zeros_like(scaled_lap)
    binary_output[(scaled_lap >= thresh_min) & (scaled_lap <= thresh_max)] = 1
    

    return binary_output   
print ('...')

This is a test bench for you to try different values easily. You can move the white dot to cahnge the threshold values. Try to find the optimal value sets and hard-code them at where the previous two functions are defined. You must finish previous part first before run it.

In [7]:

def fineTuning(thresh_min1,thresh_max1,thresh_min2,thresh_max2,thresh_min3,thresh_max3):
    rawImage = cv2.imread("./RightHookTestImage/rh3.png")
    warpedImage, M, Minv = warpImage(rawImage)
    
    ## TODO
    #To test different function you write, change the function name 
    #newImg = laplacian(warpedImage, thresh_min1, thresh_max1) #comment out this line when test sobel
    newImg = sobel(warpedImage, thresh_min1, thresh_max1) #comment out this line when test laplacian
    ####
    
    f = plt.figure(figsize= (20,15))
    f.add_subplot(1, 2, 1)
    plot1 = cv2.cvtColor(warpedImage,cv2.COLOR_BGR2RGB)
    plt.imshow(plot1)
    f.add_subplot(1, 2, 2)
    plt.imshow(newImg, cmap='gray')
    plt.show()
    
interact(fineTuning, thresh_min1 = (0, 255, 1), thresh_max1 = (0,255, 1), thresh_min2 = (0,255, 1), thresh_max2 = (0, 255, 1), thresh_min3 = (0, 255, 1), thresh_max3 = (0, 255, 1))

A Jupyter Widget

<function __main__.fineTuning>

Besides gradient method, we can also utilize the information that lane markings in united states are normally white and yellow. So we can convert the color image into a binary image, where white and yellow pixels are "1" and other colors are "0". You may use diffrent color space (RGB, LAB, HSV, HSL, YUV...), different channels and compare the effect. <br>
This link might be helpful: 
https://docs.opencv.org/3.4.3/da/d97/tutorial_threshold_inRange.html

In [8]:
def colorThres(img, thresh_min1=0,thresh_max1=255,thresh_min2=0,thresh_max2=255,thresh_min3=0,thresh_max3=255):
    # use the threshold values you found to replace 0 and 255 above
    #1. Gaussian blur the image (cv2.GaussianBlur())
    #2. Choose the color space that you like, convert image to it (cv2.cvtColor())
    #3. Apply threshold on the image to create binary image output, you might need two threshold for yellow and white
    #4. Combine results from different color spaces 
    
    ##TODO
    
    #name your result to be:
    binary_output = 
    ####
    
    return binary_output

Similar to the previous test bench. You need to find one set of color threshold at a time. 

In [9]:
def fineTuning(thresh_min1,thresh_max1,thresh_min2,thresh_max2,thresh_min3,thresh_max3):
    rawImage = cv2.imread(testImageRoot)
    rawImage = cv2.cvtColor(rawImage,cv2.COLOR_BGR2RGB)
    warpedImage, M, Minv = warpImage(rawImage)
    newImg = colorThres(warpedImage,thresh_min1,thresh_max1,thresh_min2,thresh_max2,thresh_min3,thresh_max3)
    f = plt.figure(figsize= (20,15))
    f.add_subplot(1, 2, 1)
    
    plt.imshow(warpedImage)
    f.add_subplot(1, 2, 2)
    plt.imshow(newImg, cmap='gray')
    plt.show()
    
interact(fineTuning, thresh_min1 = (0, 255, 1), thresh_max1 = (0,255, 1), thresh_min2 = (0,255, 1), thresh_max2 = (0, 255, 1), thresh_min3 = (0, 255, 1), thresh_max3 = (0, 255, 1))

A Jupyter Widget

<function __main__.fineTuning>

Now put your most fine-tuned and effective methods into this function. We will put it in our final image processing pipeline later.

In [10]:
def combinedEdgeDetection(img):
    ##TODO
    #you don't have to use all of them, comment what you don't need
    LapOutput = laplacian(img)
    SobelOutput = sobel(img)
    ColorOutput = colorThres(img)
    
    #####
    binaryImage = np.zeros_like(ColorOutput)
    #here you can use as many methods as you want. 
    binaryImage[(ColorOutput==1)|(SobelOutput==1)] = 1
    return binaryImage

Test on all 6 images. Your Result should look like:
<img src="./ResultImages/2.png">

In [8]:
images = glob.glob("./RightHookTestImage/rh*.png")
for frame in images:
    img = cv2.imread(frame)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    warpedImg, M, Minv = warpImage(img)
    edgeDetectedImg = combinedEdgeDetection(warpedImg)
    
    f = plt.figure(figsize= (20,15))
    f.add_subplot(1, 2, 1)
    plt.imshow(warpedImg)
    f.add_subplot(1, 2, 2)
    plt.imshow(edgeDetectedImg,cmap='gray')
    plt.show()

### Sliding Window Summation & Convolution
With a binary bird view image, now we need to extract out the locations of two lane markings, and plot two clear lines that covers those locations. Since the lines tend to be vertical, we can slice the image into multiple horizontal layers. For each layer, we find the most dense location. <br>

Convolution is widely used in deep learning neural networks and signal processing. It works well for feature extraction. Here we can use it to find the most likely position of the center of lane markings. Convolution between two finite sequences is just the summation of products. $$(f*g)[n] = \sum_{m=-\infty}^{\infty} f[m]\cdot g[n-m]= f[n-m] \cdot g[m]$$
1. First, we need to define the height and width of sliding window and create a 1-D array of 1s to represent it. The length of array is just the length of sliding window. 
2. Then, we start from the bottom half of the rows of pixels. To find the starting points for left and right lane markings, we sum up the lower half of each column and store the sums into an 1-D array with length equals to image width. PS: use np.sum()
3. Now we can convolute the sliding window with the array of sums. Then pick the x coordinate where the convolution result is maximum, using np.convolve() and np.argmax(). We shall search from bottom to top. In the bottom area, which is the closest, we can assume that horizontal coordinates of two lines are within certain range.
4. For the other layers, we repeat the same process. But instead of sum up half of the column, we only look at a vertical piece with height equals to sliding window height for each layer. Between 2 consecutive layers, the x-coordinates of the lane markings should be relatively close. So we assume their difference to be less than a certain margin. <br>

Optional:
* Lane markings may not always be continuous. So we can assume that the lane curvature are unchanged for layers where no obvious lines can be found. 
* If the convolution result is smaller than a certain threshold, we can consider it as background noise and set it to 0.

In [12]:
# window settings, can be changed if you want. 
window_width = 30 
window_height = rawImg.shape[0] / 10.0 # Break image into 10 vertical layers 
margin = 80 # How much to shift left or right from last found line segment for searching
window = np.ones(window_width) # Create our window template that we will use for convolutions
convThres = 100 # filtered out back ground noise

#Parameters for finding the start points 
verticle_ratio = 1.0/2  
horizontal_ratio = 0.15

def findConvCenter(image):
    
    image = np.uint8(255*image/np.max(image))

    #windowCentroids = [] #Output array that store the (left,right) window centroid positions per level
    leftCentroids = []
    rightCentroids = []

    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    ## TODO
               
    ####
    # Add what we found for the first layer
    leftCentroids.append((leftCenterX, leftCenterY))
    rightCentroids.append((rightCenterX, rightCenterY))

    #Count how many empty layers for left and right line
    leftEmptyCount, rightEmptyCount = 0, 0
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        ## TODO
        
        ####
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        ## TODO
        
        ####   
        # Find the best right centroid by using past right center as a reference
        #if last layer not found, extend margin and reuse old refrence
        ## TODO
        
        ####
        # Add what we found for that layer
        ## TODO

        ####
    ####
    #return two list of tuples 
    return leftCentroids, rightCentroids

print ('...')

...


This part is provided for you. It will draw the predicted lines on warped (bird view) image. Your result image should look like:
<img src="./ResultImages/3.png">

In [13]:
def window_mask(width, height, img_ref, centerX, centerY):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-centerY-height/2):int(img_ref.shape[0]-centerY+height/2),max(0,int(centerX-width/2)):min(int(centerX+width/2),img_ref.shape[1])] = 1
    return output

def plotWindows(warped):
    leftCentroids, rightCentroids= findConvCenter(warped)
    
    # Points used to draw all the left and right windows
    r_points = np.zeros_like(warped)
    l_points = np.zeros_like(warped)    
    
    if len(leftCentroids) > 0 and len(rightCentroids) > 0:
        if len(leftCentroids) > 0:
            # Go through each level and draw the windows 
            for level in range(0,len(leftCentroids)):
                # Window_mask is a function to draw window areas
                l_mask = window_mask(window_width,window_height,warped,leftCentroids[level][0],leftCentroids[level][1])
                # Add graphic points from window mask here to total pixels found 
                l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255

        if len(rightCentroids) > 0:
            # Go through each level and draw the windows 
            for level in range(0,len(rightCentroids)):
                # Window_mask is a function to draw window areas
                r_mask = window_mask(window_width,window_height,warped,rightCentroids[level][0],rightCentroids[level][1])
                # Add graphic points from window mask here to total pixels found 
                r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
    #If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    return output

In [1]:
images = glob.glob("./RightHookTestImage/rh*.png")#("test_images/test*.jpg")
for frame in images:
    img = cv2.imread(frame)
    #undistortedImg = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    warpedImg, M, Minv = warpImage(img)
    edgeDetectedImg = combinedEdgeDetection(warpedImg)
    plotedLinesImg = plotWindows(edgeDetectedImg)
    
    f = plt.figure(figsize= (20,15))
    f.add_subplot(1, 2, 1)

    plt.imshow(warpedImg)
    f.add_subplot(1, 2, 2)
    plt.imshow(plotedLinesImg)
plt.show()

### Solve the Equation 
We can simplfy the two lane markings as two first-order polynomial equations in the form of: $$y = a*x^2 + b*x + c$$ <br>
Luckily, we can use np.polyfit to solve the equation. The non-zero output coordinates from previous part will be the input for this function. Then use cv2.fillPoly to draw the line back to warped image.

In [15]:
def drawPoly(originImg, warpImg, leftCentroids, rightCentroids, Minv):
    #Generate polynomial functions
    ##TO DO
    
    ####
    
    # Generate y values for plotting
    plotY = np.linspace(0, warpImg.shape[0]-1, warpImg.shape[0])
    # Generate X values for polynomial line 
    ## TO DO
    leftPlotX = 
    rightPlotX = 
    
    ####
    
    #filter out the out of boundary points
    ##TO DO
    
    ####
    
    #Generate (X,Y) points for plotting
    leftPts = np.array([np.transpose(np.vstack([leftPlotX, plotY]))])
    rightPts = np.array([np.flipud(np.transpose(np.vstack([rightPlotX, plotY])))])
    points = np.hstack((leftPts, rightPts))

    #plot to warped image
    ##To DO
    
    ####
    
    #return the new image and the coefficients of two polynomial equations 
    return newImg, leftPlotX, rightPlotX

print ('...')

...


Just like before, we need to test our implementation before use it on our final pipeline. Draw the lane markings on original image after another perspective transformation. You don't need to change anything here. Your result should be similar to this:
<img src="./ResultImages/4.png">

In [9]:
images = glob.glob("./RightHookTestImage/rh*.png")
for frame in images:
    img = cv2.imread(frame)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    warpedImg, M, Minv = warpImage(img)
    edgeDetectedImg = combinedEdgeDetection(warpedImg)
    leftCentroids, rightCentroids = findConvCenter(edgeDetectedImg)
    plotedLinesImg, leftX, rightX = drawPoly(img, warpedImg, leftCentroids, rightCentroids, Minv)
    
    f = plt.figure(figsize= (20,15))
    f.add_subplot(1, 2, 1)
    plt.imshow(img)
    f.add_subplot(1, 2, 2)
    plt.imshow(plotedLinesImg)
plt.show()

Combine the functions to get the pipline. 

In [17]:
def imagePipeline(img):
    img = cv2.resize(img, (1080, 720))
    warpedImg, M, Minv = warpImage(undistortedImg)
    edgeDetectedImg = combinedEdgeDetection(warpedImg)
    leftCentroids, rightCentroids = findConvCenter(edgeDetectedImg)
    imgOut, leftX, rightX = drawPoly(undistortedImg, warpedImg, leftCentroids, rightCentroids, Minv)
    return imgOut
    
print ('...')

...


Congratulations! It is a great achievement to build a Lane Detection System! Now we can to test our code in RightHook Simulator. See detailed instruction on our website:
https://publish.illinois.edu/safe-autonomy/ <br>
Also, don't forgot the report! You need to submit the finished jupyter file (.ipynb) with the results and the report (pdf) to Compass 2g. Please submit a zip file with name "netid1_netid2_mp1". <br>
Finally, most importantly, take a photo of your simulation result and post it on your Facebook so your friends know how cool this course is!